In [1]:
import pandas as pd
import sqlite3
import os

In [ ]:
files_tables = {
    "orders.csv": "orders",
    "products.csv": "products",
    "aisles.csv": "aisles",
    "departments.csv": "departments",
    "order_products__prior.csv": "order_products_prior",
    "order_products__train.csv": "order_products_train"
}

# Connect to database
db_name = "instacart.db"
conn = sqlite3.connect(db_name)

In [3]:
print("--- Starting Import ---")
for file, table in files_tables.items():
    if os.path.exists(file):
        print(f"Reading {file}...")
        chunksize = 100000 
        for i, chunk in enumerate(pd.read_csv(file, chunksize=chunksize)):
            chunk.to_sql(table, conn, if_exists='append', index=False)
            if i == 0: print(f"  - Started writing to '{table}'...")
        print(f"Finished loading {table}")
    else:
        print(f"File not found: {file}")

print("\n--- Database Ready ---")

def run_query(q):
    return pd.read_sql_query(q, conn)

--- Starting Import ---
Reading orders.csv...
  - Started writing to 'orders'...
Finished loading orders
Reading products.csv...
  - Started writing to 'products'...
Finished loading products
Reading aisles.csv...
  - Started writing to 'aisles'...
Finished loading aisles
Reading departments.csv...
  - Started writing to 'departments'...
Finished loading departments
Reading order_products__prior.csv...
  - Started writing to 'order_products_prior'...
Finished loading order_products_prior
Reading order_products__train.csv...
  - Started writing to 'order_products_train'...
Finished loading order_products_train

--- Database Ready ---


# Analysis: Most Popular Produce Item

Identify the single most purchased item specifically from the **"produce"** department.

1. **Join** `order_products_prior` (sales) with `products` (names) and `departments` (categories).
2. **Filter** specifically for the `'produce'` department.
3. **Count** the frequency of each product and **Sort** descending.

In [4]:
query = """
SELECT 
    p.product_name, 
    COUNT(*) as purchase_count
FROM 
    order_products_prior op
JOIN products p ON op.product_id = p.product_id
JOIN departments d ON p.department_id = d.department_id
WHERE 
    d.department = 'produce'
GROUP BY 
    p.product_name
ORDER BY
    purchase_count DESC
LIMIT 1;
"""

run_query(query)

,product_name,purchase_count
0,Banana,12759255


# Market Basket Analysis: What is bought with Bananas?

**Objective:** Top 10 products most frequently purchased in the same transaction as Bananas (Product ID: `24852`). 


In [5]:
query = """
SELECT 
    p.product_name, 
    COUNT(*) as frequency
FROM order_products_prior op
JOIN products p ON op.product_id = p.product_id
WHERE op.order_id IN (
    SELECT order_id 
    FROM order_products_prior 
    WHERE product_id = 24852
)
AND op.product_id != 24852 
GROUP BY p.product_name
ORDER BY frequency DESC
LIMIT 10;
"""

run_query(query)

,product_name,frequency
0,Organic Strawberries,505404
1,Organic Avocado,480555
2,Organic Baby Spinach,462555
3,Strawberries,371088
4,Large Lemon,367920
5,Organic Fuji Apple,305487
6,Cucumber Kirby,288873
7,Limes,285948
8,Organic Whole Milk,284769
9,Organic Hass Avocado,280998


# The "Most Addictive" Products (Reorder Rate)

### 🔁 Customer Loyalty: The "Most Addictive" Products

**Business Question:** Which products have the highest customer loyalty? If someone buys it once, what is the probability they will buy it again?

### 📊 SQL Query
To find this, we calculate the **Reorder Rate** (Total Reorders / Total Purchases) for each product. Then we filtered for products bought at least 100 times to remove statistical anomalies (like a product bought only once and reordered once having a 100% rate).

In [7]:
query = """
SELECT 
    p.product_name,
    COUNT(op.order_id) as total_purchases,
    SUM(op.reordered) * 1.0 / COUNT(op.order_id) as reorder_rate
FROM order_products_prior op
JOIN products p ON op.product_id = p.product_id
GROUP BY p.product_name
HAVING total_purchases > 100
ORDER BY reorder_rate DESC
LIMIT 10;
"""

run_query(query)

,product_name,total_purchases,reorder_rate
0,Raw Veggie Wrappers,612,0.941176
1,Serenity Ultimate Extrema Overnight Pads,783,0.931034
2,Orange Energy Shots,117,0.923077
3,Chocolate Love Bar,909,0.920792
4,Soy Powder Infant Formula,315,0.914286
5,Simply Sleep Nighttime Sleep Aid,405,0.911111
6,"Energy Shot, Grape Flavor",198,0.909091
7,Sparking Water,540,0.900000
8,Russian River Valley Reserve Pinot Noir,270,0.900000
9,Maca Buttercups,900,0.900000


# Time Analysis: Peak Shopping Hours

**Business Question:** What time of day do people buy the most groceries? 

Insight from this can help optimize server maintenance windows and target time-based ads.

### 📊 SQL Query
Grouped the `orders` table by the `order_hour_of_day` column and count the total number of orders for each hour.

In [6]:
query = """
SELECT 
    order_hour_of_day,
    COUNT(order_id) as total_orders
FROM 
    orders
GROUP BY order_hour_of_day
ORDER BY order_hour_of_day;
"""

run_query(query)

,order_hour_of_day,total_orders
0,0,68274
1,1,37194
2,2,22617
3,3,16422
4,4,16581
5,5,28707
6,6,91587
7,7,275604
8,8,534603
9,9,773436


# Order Metrics: Average Cart Size

**Business Question:** How many items does a typical customer put in their basket per order? 

### 📊 SQL Query
We use a **Common Table Expression (CTE)** to first count the number of items in *each individual order*. Then, the main query calculates the overall average of those counts.

In [8]:
query = """

WITH OrderSizes AS (
    SELECT order_id, COUNT(product_id) as items_in_cart
    FROM order_products_prior
    GROUP BY order_id
)
SELECT AVG(items_in_cart) as average_cart_size
FROM OrderSizes;
"""

run_query(query)

,average_cart_size
0,30.26665


# Department vs. Department (Volume Analysis)

**Business Question:** Which grocery departments drive the most sales volume for Instacart?

### 📊 SQL Query
We join the `order_products_prior` table with the `products` and `departments` tables to aggregate the total number of items sold per department.

In [10]:
query = """
SELECT 
    d.department, 
    COUNT(op.product_id) as total_items_sold
FROM order_products_prior op
JOIN products p ON op.product_id = p.product_id
JOIN departments d ON p.department_id = d.department_id
GROUP BY d.department
ORDER BY total_items_sold DESC;
"""

run_query(query)

,department,total_items_sold
0,produce,255940857
1,dairy eggs,146178432
2,snacks,77963850
3,beverages,72633483
4,frozen,60383664
5,pantry,50640579
6,bakery,31773249
7,canned goods,28837566
8,deli,28383723
9,dry goods pasta,23398929
